# KNN介绍

k近邻法(k-nearest neighbors)是由Cover和Hart于1968年提出的，它是懒惰学习(lazy learning)的著名代表。<br>它的工作机制比较简单：
- 给定一个测试样本
- 计算它到训练样本的距离
- 取离测试样本最近的`k`个训练样本
- “投票法”选出在这k个样本中出现最多的类别，就是预测的结果


> 距离衡量的标准有很多，常见的有：欧氏距离、曼哈顿距离、余弦值、Lp距离(Lp范数)。

什么意思呢？先来看这张图

![2个类](img\P1.png)

我们对应上面的流程来说
- 1.给定了红色和蓝色的训练样本，绿色为测试样本
- 2.计算绿色点到其他点的距离
- 3.选取离绿点最近的k个点
- 4.选取k个点中，同种颜色最多的类。例如：k=1时，k个点全是蓝色，那预测结果就是Class 1；k=3时，k个点中两个红色一个蓝色，那预测结果就是Class 2

## 举例

这里用`欧氏距离`作为距离的衡量标准，用鸢尾花数据集举例说明。
鸢尾花数据集有三个类别，每个类有150个样本，每个样本有4个特征。

先来回顾一下欧氏距离的定义(摘自维基百科)：<br>
> 在欧几里得空间中，点 x = (x1,...,xn) 和 y = (y1,...,yn) 之间的欧氏距离为<br>
 $d(x,y):={\sqrt  {(x_{1}-y_{1})^{2}+(x_{2}-y_{2})^{2}+\cdots +(x_{n}-y_{n})^{2}}}={\sqrt  {\sum _{{i=1}}^{n}(x_{i}-y_{i})^{2}}}$<br>
 
> 向量 ${\displaystyle {\vec {x}}}$的自然长度，即该点到原点的距离为<br>
 $\|{\vec  {x}}\|_{2}={\sqrt  {|x_{1}|^{2}+\cdots +|x_{n}|^{2}}}$<br>
 它是一个纯数值。在欧几里得度量下，两点之间线段最短。

现在给出六个训练样本，分为三类，每个样本有4个特征，编号为7的`名称`是我们要预测的。

|编号|花萼长度(cm)|花萼宽度(cm)|花瓣长度(cm)|花瓣宽度(cm)|名称|
|----|----------|-----------|-----------|----------|----|
|1|4.9|3.1|1.5|0.1|Iris setosa|
|2|5.4|3.7|1.5|0.2|Iris setosa|
|3|5.2|2.7|3.9|1.4|Iris versicolor|
|4|5.0|2.0|3.5|1.0|Iris versicolor|
|5|6.3|2.7|4.9|1.8|Iris virginica|
|6|6.7|3.3|5.7|2.1|Iris virginica|
|7|5.5|2.5|4.0|1.3|    ?    |

按照之前说的步骤，我们来计算测试样本到各个训练样本的距离。例如到第一个样本的距离$d_{1}=\sqrt{(4.9 - 5.5)^2 + (3.1 - 2.5)^2 + (1.5 - 4.0)^2 + (0.1 - 1.3)^2} = 2.9$

写一个函数来执行这个操作吧

In [5]:
import numpy as np
def calc_distance(iA,iB):
    temp = np.subtract(iA, iB)      # 对应元素相减
    temp = np.power(temp, 2)        # 元素分别平方
    distance = np.sqrt(temp.sum())  # 先求和再开方
    return distance

testSample = np.array([5.5, 2.5, 4.0, 1.3])
print("Distance to 1:", calc_distance(np.array([4.9, 3.1, 1.5, 0.1]), testSample))
print("Distance to 2:", calc_distance(np.array([5.4, 3.7, 1.5, 0.2]), testSample))
print("Distance to 3:", calc_distance(np.array([5.2, 2.7, 3.9, 1.4]), testSample))
print("Distance to 4:", calc_distance(np.array([5.0, 2.0, 3.5, 1.0]), testSample))
print("Distance to 5:", calc_distance(np.array([6.3, 2.7, 4.9, 1.8]), testSample))
print("Distance to 6:", calc_distance(np.array([6.7, 3.3, 5.7, 2.1]), testSample))

Distance to 1: 2.9
Distance to 2: 2.98496231132
Distance to 3: 0.387298334621
Distance to 4: 0.916515138991
Distance to 5: 1.31909059583
Distance to 6: 2.36854385647


如果我们把k定为3，那么离测试样本最近3个依次是：

|编号|名称|
|----|---|
|3|Iris versicolor|
|4|Iris versicolor|
|5|Iris virginica|

显然测试样本属于`Iris versicolor`类的“票数”多一点，事实上它的确属于这个类。

## 缺点

看到这你可能已经意识到有点问题了，选择一个合适的k值的确可以减少模型受噪音的音响，但是当样本不平衡的时候呢？

![不平衡](img\P2.png)

我们从直观上可以看出X和Y应该属于$\omega_{1}$，但是对于Y来说，在k范围内，更多的点属于$\omega_{2}$，这就造成了错误分类。<br>
由于它属于懒惰学习，因此需要大量的空间来存储训练实例，在预测时它还需要与已知所有实例进行比较，这就造成了它的复杂度比较高。

## 一个结论

在周志华编著的《机器学习》中证明了最近邻学习器的泛化错误率不超过贝叶斯最优分类器的错误率的两倍，在原书的226页，这里就不摘抄了。

## 代码实现KNN算法

``` Python
import pandas as pd
import numpy as np
import random

def load_data(path, rate = 0.8):
    ''' Making dataSet
    Args:
        path: DataSet file path.
        rate: TrainSet percentage.
    Returns:
        trainSet: Training set.
        trainLabel: Training label.
        testSet: Test set.
        testLabel: Test label.
    '''
    data = pd.read_csv(path)
    randIndex = random.sample(range(0, len(data)), len(data))
    trainSet = data.loc[randIndex[:int(len(data) * rate)]]
    testSet = data.loc[randIndex[int(len(data) * rate):]]
    trainLabel = trainSet['name']
    testLabel = testSet['name']
    trainSet.drop('name', axis=1, inplace=True)
    testSet.drop('name', axis=1, inplace=True)
    return np.array(trainSet), np.array(trainLabel), np.array(testSet), np.array(testLabel)

def euclideanDistance(instance1, instance2):
    ''' Calculate two-point Euclidean distance
    Args:
        instance1: Numpy array type training set.
        instance2: The element of test set.
    Returns:
        distance: The distance of instance2 to instance1, the type is list.
    '''
    distance = []
    for instance in instance1:
        temp = np.subtract(instance, instance2)
        temp = np.power(temp, 2)
        temp = temp.sum()
        distance.append(np.sqrt(temp))
    return distance

def getNeighbors(distance, label, k):
    ''' Calculate k-nearest element
    Args:
        distance: Return by euclideanDistance function.
        label: Training label.
        k: k-nearest
    Returns:
        neighbors: The k-nearest elements.
    '''
    df = pd.DataFrame()
    df['distance'] = distance
    df['label'] = label
    neighbors = df.sort_values(by='distance').reset_index()
    return neighbors.loc[:k-1]

def getResult(neighbors):
    neighbors = neighbors['label'].value_counts()
    neighbors = pd.Series(neighbors).sort_values(ascending=False).reset_index()
    return neighbors['index'][0]

def getAccuracy(testData, testLabel, trainSet, trainLabel):
    ''' Calculate accuracy percentage.
    Args:
        testData: Numpy array of test set.
        testLabel: Numpy array of test labels.
        trainSet: Numpy array of train set.
        trainLabel: Numpy array of train labels.
    Returns:
        score/len(trainLabel)
    '''
    score = 0
    i = 0
    for instance in testData:
        distance = euclideanDistance(trainSet, instance)
        neighbors = getNeighbors(distance, trainLabel, 5)
        result = getResult(neighbors)
        if result == testLabel[i]:
            score += 1
        i += 1
    return score/len(trainLabel)

if  __name__ == "__main__":
    trainData, trainLabel, testData, testLabel = load_data("irisdata.txt", 0.67)
    print(getAccuracy(np.array(testData), np.array(testLabel), np.array(trainData), np.array(trainLabel)))

```

代码写的比较糟糕，知道怎么个流程就好。